Integración

Sean los siguientes datasets:

Información demográfica por cuidades de EEUU: https://public.opendatasoft.com/explore/dataset/us-cities-demographics/table/?sort=total_population

Temperaturas por ciudad: https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data

Observemos que podemos procesar y combinar los datos usando pandas para luego responder a preguntas analíticas.

Vamos a realizar un pequeño ejercicio de la preparación de los datos:

In [ ]:
#Cargamos los datos de temperaturas y demografía

import pandas as pd

demographics = pd.read_csv("us-cities-demographics.csv", sep=";")
temperature = pd.read_csv("archive/GlobalLandTemperaturesByCity.csv", sep=",")

In [ ]:
#Seleccionamos sólo las ciudades de EEUU

usa_temps = temperature[temperature["Country"] == "United States"]
usa_temps.head()

In [ ]:
#Calculamos temperaturas: mínimas, máximas y promedio por ciudad
aggregated_temps = (
    usa_temps
    .groupby(["City", "Latitude", "Longitude"])
    .agg({"AverageTemperature": ["min", "max", "mean"]}, skipna=True)
)

aggregated_temps = aggregated_temps.round(2).reset_index().sort_values("City")

aggregated_temps.columns = aggregated_temps.columns.map(lambda x: x[0] if x[1] == "" else x[1])

In [ ]:
#Aplicamos agregaciones para estudiar la pregunta: las ciudades con mayor población tienen en promedio temperaturas altas o bajas?

# Seleccionamos las colúmnas importantes de los datos demográficos
subset_dems = demographics[["City", "Total Population"]].copy()
subset_dems.drop_duplicates(inplace=True)

merged = pd.merge(aggregated_temps, subset_dems, on="City")

# Agrupamos en categorías por rangos de valores
agg_df = merged.copy()
agg_df['Population_Concentration'] = pd.cut(
    agg_df['Total Population'],
    bins=[0, 100000, 500000, 1000000, float('inf')],
    labels=['Low', 'Moderate', 'High', 'Very High']
)

agg_df['Temperature_Category'] = pd.cut(
    agg_df['mean'],
    bins=[float('-inf'), 10, 20, 30, float('inf')],
    labels=['Very Cold', 'Cold', 'Warm', 'Hot']
)

# Aplicamos una ponderación para ordenar por categoría de concentración
weights = {
    'Low': 1,
    'Moderate': 2,
    'High': 3,
    'Very High': 4
}

weighted_vals = agg_df['Population_Concentration'].map(weights)
agg_df.sort_values("Population_Concentration", ascending=False).head(10)

La tabla resultante es:

City	Latitude	Longitude	min	max	mean	Total Population	Concentration	Temperatura
Houston	29.74N	96.00W	4.51	31.52	20.23	2298628	Very High	Warm
Phoenix	32.95N	112.02W	5.77	34.38	21.05	1563001	Very High	Warm
San Jose	37.78N	122.03W	4.40	22.46	14.45	1026919	Very High	Cold
San Diego	32.95N	117.77W	8.93	23.11	16.12	1394907	Very High	Cold
San Antonio	29.74N	97.85W	4.02	32.17	19.98	1469824	Very High	Cold
Los Angeles	34.56N	118.70W	4.02	27.34	15.88	3971896	Very High	Cold
New York	40.99N	74.56W	-9.16	29.14	9.52	8550405	Very High	Very Cold
Chicago	42.59N	87.27W	-8.59	27.80	10.07	2720556	Very High	Cold
Dallas	32.95N	96.70W	-0.07	33.74	18.06	1300082	Very High	Cold
Philadelphia	39.38N	74.91W	-5.67	30.45	11.86	1567442	Very High	Cold

In [ ]:
La tabla resultante es:

City	Latitude	Longitude	min	max	mean	Total Population	Concentration	Temperatura
Houston	29.74N	96.00W	4.51	31.52	20.23	2298628	Very High	Warm
Phoenix	32.95N	112.02W	5.77	34.38	21.05	1563001	Very High	Warm
San Jose	37.78N	122.03W	4.40	22.46	14.45	1026919	Very High	Cold
San Diego	32.95N	117.77W	8.93	23.11	16.12	1394907	Very High	Cold
San Antonio	29.74N	97.85W	4.02	32.17	19.98	1469824	Very High	Cold
Los Angeles	34.56N	118.70W	4.02	27.34	15.88	3971896	Very High	Cold
New York	40.99N	74.56W	-9.16	29.14	9.52	8550405	Very High	Very Cold
Chicago	42.59N	87.27W	-8.59	27.80	10.07	2720556	Very High	Cold
Dallas	32.95N	96.70W	-0.07	33.74	18.06	1300082	Very High	Cold
Philadelphia	39.38N	74.91W	-5.67	30.45	11.86	1567442	Very High	Cold